# GP Cartpole Server

This notebook introduces the genetic programming (GP) approach for the cartpole REST interface. GP creates a program for the interface's default controllers. It enables the cartpole REST interface to reply with a concrete response to a client's requests.

The figure illustrates the position of the GP software within the REST interface implementation.  

![REST gp design](http://www.plantuml.com/plantuml/png/5SWx3i8m303Ggy05uW4cPjYO2dlLEbOQ9R4ZVyJrKT_uhfMnHwRGVZdSfpq0TFkbKls8FOmKXPtO4ye0p8Jjfl1StlVbzVd2sCn9WSLf5sa6vwvbhfci5aun-Xy0)

The [OpenAPI generator](https://github.com/OpenAPITools/openapi-generator) creates a server stub consisting of the REST interface and the REST controller prototype using the [cartpole API spec](https://raw.githubusercontent.com/cdeck3r/DRL4REST/master/src/cartpole/spec/OpenAPIv1.json). In this approach we replace the controller's empty function body with an implementation found through genetic programming. Additionally, we introduce a ServerModel, which keeps a state if required. One may access the state using CRUD operations. 

In contrast to mocking solutions, e.g. [Prism](https://github.com/stoplightio/prism), GP enables more complex behavior than just a compliant response to a client's request.

## Example: `cart_get()` Controller Function

As a example for the REST controller, look at the `cart_get()` controller prototype the OpenAPI generator created for the cartpole REST interface. The generator nicely separates the controller from all the marshalling and unmarshalling operations for data transport from/to the client.

```python
def cart_get():  # noqa: E501
    """cart_get

     # noqa: E501


    :rtype: Cart
    """
    return 'do some magic!'
```

A concrete example implementation would look like the following:

```python
def cart_get():  # noqa: E501
    # just return a Cart object with random values

    position = random()
    velocity = random()
    direction = choice(["left", "right"]) 

    return Cart(position=position, velocity=velocity, direction=direction)
```

The `Cart` class is one the generated models. The constructor defines the params as follows:
```python
def __init__(self, position=None, velocity=None, direction=None):  # noqa: E501
    """Cart - a model defined in OpenAPI

    :param position: The position of this Cart.  # noqa: E501
    :type position: float
    :param velocity: The velocity of this Cart.  # noqa: E501
    :type velocity: float
    :param direction: The direction of this Cart.  # noqa: E501
    :type direction: str
    """
    ...
```

## GP Problem Formulation

For the `cart_get()` example from above, we envision the following controller implementation: if there is already a cart object, then the controller will return this object. Otherwise, the controller creates a new one and returns it. Here's the pseudo code:

```python
if cart exists 
    return cart
else
    return create_cart()
```
**The GP problem is: Synthesize a program with a RESTful behavior.**

We test run the synthesized code within the `evaluate()` function of the GP algorithm. It is successfull, if

1. `cart_get()` controller runs without any exception 
1. `cart_get()` controller returns a non-null result 
1. `cart_get()` controller returns a non-empty result 
1. repeated `cart_get()` requests result in the same response

The `evaluate()` function scores each GP created programs to report the fitness for the program evolution.

## Import libs

DEAP was previously installed in this image. If not, run `pip install deap`.

In [1]:
import random

import numpy

from functools import partial

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp

The OpenAPI generated interface code is based on python-flask. If necessary, activate the following cell `ESC + y` and install the requirements.

In [2]:
%%bash
# define project environment
PROJECT_DIR="/DRL4REST"
OPENAPI_SERVER_DIR="$PROJECT_DIR/openapi/cartpole/python-flask"

# install requirements
cd "$OPENAPI_SERVER_DIR" || exit
pip install -r requirements.txt || exit
pip install -r test-requirements.txt || exit

# install other requirements
pip install Werkzeug==0.16.1 || exit

Ignoring connexion: markers 'python_version == "3.5"' don't match your environment
Ignoring connexion: markers 'python_version == "3.4"' don't match your environment
Ignoring connexion: markers 'python_version <= "2.7"' don't match your environment


In [3]:
# set path to import code generated by OpenAPI
import sys
sys.path.append("/DRL4REST/openapi/cartpole/python-flask")

## GP Software Workflow

The OpenAPI generator creates default controllers and tests. We replace the default controller by the GP one, called `GP_Controller`, and extend the default tests by `GP_TestDefaultController`. The following activity diagram displays how these parts are embedded within the actions of the genetic programming framework.

![GP software activity diagram](http://www.plantuml.com/plantuml/png/1S4zhi8m303WgxntWNY0YHd4me0AUvKwLciriP5_GDoUx_jdmPwDqlhV5l7o4u3LJravjbpIoL0bIAAW3i16k1yCpjFjE5qUJnZLWmn4WnRLtU5wdnivhp9N37vpV4kE_WC0)

## ServerModel CRUD Functions

`CartpoleServer` is the implementation of the ServerModel. The ServerModel is a static class. The GP controller utilizes the model to store state between successive calls. The ServerModel provides CRUD functions for state access and manipulation.

In [4]:
from openapi_server.models import Cart

class CartpoleServer(object):

    # state as class variable
    _cart = None

    # set the class in a defined (initial) state
    @classmethod
    def reset(cls):
        cls._cart = None
    
    @classmethod
    def create_cart(cls):
        position = random.random()
        velocity = random.random()
        # we know the values from the models, e.g.
        #   models/cart.py or models/direction.py
        direction = random.choice(["left", "right"])
        cls._cart = Cart(position=position, velocity=velocity, direction=direction)

    @classmethod
    def read_cart(cls):
        return cls._cart
    
    @classmethod
    def update_cart(cls):
        if isinstance(cls._cart, Cart):
            cls._cart.direction='left'

    @classmethod
    def delete_cart(cls):
        cls._cart = None

## Monkey Patching

Monkey patching changes the default behavior of a piece of code at runtime without changing its original source. For example, we want to change the behavior of the default controller function prototype `cart_get()`. Monkey patching replaces the prototype by the GP controller. When running the testcases, the GP controller gets executed instead of the default controller.

Concretely, the view function of an REST endpoint is patched to run a new controller function other than the default controller. While the openapi generator keeps controller nicely separated from the Connexion / Flask framework, it wraps them when loading the openapi spec. Basically, it applies various Python [decorators within the Operation object](https://github.com/zalando/connexion/blob/bed4b95205205861bd6014282d0f61d50231d3ac/connexion/operations/abstract.py#L340) around the controller function to add additional behavior. Note, [do not confuse it with the decorator pattern](https://wiki.python.org/moin/DecoratorPattern). As a consequence, the monkey patching approach has to unwrap the enpoint's view function at first to re-gain access to the (default) controller function. Afterwards, it replaces the code. 

In [5]:
class MonkeyPatching:
    def __init__(self, app=None):
        self._rules = []
        self._app = app
    
    @property
    def app(self):
        return self._app
    
    @app.setter
    def app(self, app):
        self._app = app
    
    # test if endpoint config exists
    def _endpoint_exists(self, url_path, method):
        self._rules = []
        
        for r in self._app.url_map.iter_rules():
            if (url_path == str(r)) and (method.upper() in r.methods):
                self._rules.append(r)
                return True
        return False

    # gets the endpoint's view function
    # returns the default controller functio
    def _unwrap(self, view_func):
        wrapped_func = view_func
        try:
            while wrapped_func.__wrapped__ :
                wrapped_func = wrapped_func.__wrapped__
        except Exception:
            # this is the default_controller_func
             return wrapped_func
    
    # replace endpoint's controller function with the one 
    # from the parameters
    def patch(self, url_path, method, controller_func):
        if self._endpoint_exists(url_path, method):
            rule = self._rules[0]
            view_func = self._app.view_functions[rule.endpoint]
            default_controller_func = self._unwrap(view_func)
            default_controller_func.__code__ = controller_func.__code__
            return True
        
        return False

## Testing the GP Controller

The OpenAPI generator creates the unittest `TestDefaultController` containing test mesthods for each default controller. The `GP_TestDefaultController` overrides these tests in order to enable additional checks of the response data. Furthermore, this class enables the Monkey Patching of the controller function. As a result, the `test_cart_get()` function from this class actually tests the GP controller function.

In [6]:
from openapi_server.test.test_default_controller import TestDefaultController
from pprint import pprint

import unittest

class GP_TestDefaultController(TestDefaultController):

    def __init__(self, mp=MonkeyPatching(), app_config_testing=True, assert_score=1000):
        super().__init__()
        self.app = super().create_app()
        self.app.config['TESTING'] = app_config_testing
        self.client = self.app.test_client()
       
        # associate with MonkeyPatching
        self.mp = mp
        self.mp.app = self.app    
        
        # score for response evaluation
        self._score = -1
        self._assert_score = assert_score
        
        self._prev_responses = []

        
    @property
    def score(self):
        return self._score
    
    @score.setter
    def score(self, s):
        if s < 0:
            self._score = -1
        else:
            self._score = s
        
    def reset_score(self):
        self.score = -1
        self._prev_responses = []
        
    # quantify the response, we use the status code as score 
    def score_response(self, resp):
        # src: https://stackoverflow.com/a/3844832
        def checkEqualEntries(iterator):
            return len(set(iterator)) <= 1
        
        # store the response body
        self._prev_responses.append(str(resp.json))
        # compare all responses 
        if len(self._prev_responses) > 1 and checkEqualEntries(self._prev_responses):
            # expect them the same, neutralize increasing score to keep them small
            self.score -= resp.status_code
        
        self.score += resp.status_code
    
    """
    Changes the behavior of a url_path with a new controller function
    
    Returns True if endpoint was found and successfully patched

    Limits: 
      * works only if method is unique for url_path 
      * does not support aliases
    """
    def endpoint_config(self, url_path, method, controller_func):
        return self.mp.patch(url_path, method, controller_func)

    # overrides the default test_cart_get()
    def test_cart_get(self):
        response = super().test_cart_get() #assert200(response)
        # add another important assertion
        self.assertEqual(response.content_type, 
                    'application/vnd.cartpole.cart+json', 
                    'Please check the openapi.yaml for unique content-type in response section'
                   )
        return response

    # safe version of test_cart_get(), i.e. avoids exceptions 
    def safe_test_cart_get(self):
        try:
            response = super().test_cart_get() #assert200(response)        
            self.assertEqual(response.content_type, 
                        'application/vnd.cartpole.cart+json', 
                        'Please check the openapi.yaml for unique content-type in response section'
                       )
        except AssertionError as error:
            # score an assertion
            self.score += self._assert_score
            return
            
        self.score_response(response)

## Genetic Program Implementation

The GP program bases upon the Artificial Ant Problem from the [DEAP example site](https://deap.readthedocs.io/en/master/examples/gp_ant.html). The implementation consists of

* generic functions for the control flow
* controller functions interacting with the ServerModel
* `evaluate()` function to report the controller's fitness

### Generic Functions

These are generic functions, e.g. for control flow and concatenating functions with each other.

In [7]:
def progn(*args):
    for arg in args:
        arg()

def prog2(out1, out2): 
    return partial(progn,out1,out2)

def prog3(out1, out2, out3):     
    return partial(progn,out1,out2,out3)

def _if_then_else(condition, out1, out2):
    if condition() is not None:
        out1()
    else:
        out2()

def if_then_else(condition, out1, out2):
    return partial(_if_then_else, condition, out1, out2)

def _if_then(condition, out1):
    if condition() is not None:
        out1()
        
def if_then(condition, out1):
    return partial(_if_then, condition, out1)

### Controller Functions

These are the functions the GP algorithm strings together utilizing the generic functions from above. The created program forms an individual and is tested by the `evaluate()` function. The controller functions utilize the CartpoleServer's CRUD functions.

In [8]:
cps = CartpoleServer
cps.reset()

In [9]:
# let's create the set of all functions DEAP shall work with
pset = gp.PrimitiveSet("MAIN", 0)
pset.addPrimitive(if_then_else, 3)
pset.addPrimitive(if_then, 2)

pset.addPrimitive(prog2, 2)
pset.addPrimitive(prog3, 3)

pset.addTerminal(cps.create_cart)
pset.addTerminal(cps.read_cart)
pset.addTerminal(cps.update_cart)
pset.addTerminal(cps.delete_cart)

In [10]:
# cart_get() controller must return something when done
# this function becomes the new root for the GP tree created from the pset above
# it runs the GP tree first and returns the Cart afterwards
def return_cart_get(gp_tree):
    gp_tree()
    return cps.read_cart

### `evaluate()` Function

For each created GP controller function we run the `evaluate()` function. It patches the endpoint with the new controller function and runs the controller unittest. Evaluation reports the controller's fitness to guide the next program evolution. The UML diagram below illustrate the relations between the participating components.

**Note:** The `GP_Controller` class wraps the GP created `_gp_controller` function. It is necessary, because the `_gp_controller` is a partial function. 

![GP evaluate class diagram](http://www.plantuml.com/plantuml/png/1S4x3i9030JGgy05rWLKr20Q2XJeqSOPaMW_HjunN9_tzblN6Wl2wsubF-CaulcNtXlNc6F0rJhHcTIQw08j1vVpT3jEbyTJMXw4IpTYDYj3hlSNlhb69bAq-WS0)

The following sequence diagram depicts the interaction between the objects.

![GP evaluate sequence diagram](http://www.plantuml.com/plantuml/png/1S4z3i8m30RGgy05uW4cPX0B0ohiLUf-ebKJ6_m3r-Uz_IhSSaPQsoqHRpyHMVsLrnfBJkamLWb85DPEF8EtWz5vk1s7o-D9lNh0I3GmgMvEr_k8RsrP0wFZKxAtFm00)

MonkeyPatching requires a function with a `__code__` section. However, the GP created controller program is a partial function, which [does not have a `__code__` section](https://stackoverflow.com/questions/56881670/partial-function-object-has-no-attribute-code). As a consequence, the `GP_Controller` static class wraps the GP created partial function into a function with a `__code__` section. 

In [11]:
# Encapsulates the gp_controller for execution
class GP_Controller:
    
    _gp_controller = None
    
    @classmethod
    def set_controller_func(cls, func):
        cls._gp_controller = func

    """ Runs the gp_controller.
    
        gp_controller_func is called from a different scope, so it needs to 
        search through all imported modules for the GP_Controller class in
        order to access the class variable _gp_controller.

        CAUTION: 
        This is fragile. It finds the first occurance of the GP_Controller
        within all imported modules.
        
        DEFAULT:
        It searches the __main__ module only for the GP_Controller class.
    """
    @staticmethod
    def gp_controller_func():
        # This is the search routine
        """
        gpc_pointer = None
        for m_name in sys.modules:
            try:
                gpc_pointer = getattr(sys.modules[m_name], 'GP_Controller')
                break
            except AttributeError:
                continue
        return gpc_pointer._gp_controller()
        """
        # finds the GP_Controller class in the __main__ module
        return getattr(sys.modules['__main__'], 'GP_Controller')._gp_controller()
    
gpc = GP_Controller

In [763]:
import types
import json

# evaluate()
def evalRESTController(individual):
   
    def add_func2pset(expr, pset, func, num_args):
        pset_cloned = toolbox.clone(pset)
        pset_cloned.addPrimitive(func, num_args)
        prim_func = gp.Primitive(func.__name__, [object], object)
        expr_func = [prim_func] + expr
        return expr_func, pset_cloned

    # we add the return_cart_get() als new root of the individual
    # togehter they form the gp_controller

    assert isinstance(return_cart_get, types.FunctionType), 'return_cart_get() not defined'
    
    # 1. dismantle the individual to get the expressions
    expr = list(individual)

    # 2. add return_cart_get als new root
    expr_return, pset_return = add_func2pset(expr=expr,
                                             pset=pset,
                                             func=return_cart_get, 
                                             num_args=1)
    # 3. rebuild individual (type: PrimitiveTree) from expressions 
    individual_return = gp.PrimitiveTree(expr_return)
    
    # 4. .. and compile tree to functional Python code 
    gp_controller = gp.compile(individual_return, pset_return)

    # store the controller
    gpc.set_controller_func(gp_controller)
    #gpc.set_controller_func(testcase_all_correct)

    # Replace the default controller with the gp_controller 
    url_path = '/api/v1/cart'
    gp_test = GP_TestDefaultController()
    ret = gp_test.endpoint_config(url_path, 'get', gpc.gp_controller_func)
    # ... and test
    cps.reset()
    gp_test.reset_score()
    
    #gp_test.test_cart_get()
    # run the test several times and expect the same as the first one
    for i in range(5):
        gp_test.safe_test_cart_get()
    
    # retrieve score 
    fitness = gp_test.score
    
    if fitness < 1999: 
        # store individual
        gp_program = str(individual)
        # store _prev_responses
        all_responses = gp_test._prev_responses
        with open("/home/jovyan/work/gp_program.json", "a") as f:
            json_gp_program = json.dumps(gp_program)
            json_all_responses = json.dumps(all_responses)
            f.write(str(fitness))
            f.write(json_gp_program)
            f.write(json_all_responses)
            f.write('\n\n')
    
    return fitness,

### Assemble and Configure the GP

We configure the GP to solve a minimization problem, `FitnessMin`. The `evaluate()` function returns the cumulative sum of the responses' status codes when testing the controller as fitness. 

Returned scored for a GP created controller when executed once:
$$
    s = \begin{cases}
    \text{assert_score, } & \, \text{if assertion occurs during the controller test} \\
    \text{status_code(response)} & \, \text{else}
    \end{cases}
$$

Returned fitness for a GP created controller when executed once or more:
$$
    fitness = \sum_{\text{controller tests per Individual}} s
$$
An `Individual` is a program. The initally generated `Individual` is a `PrimitiveTree` object created from the expressions in `pset` by the `gp.genFull` function. You may call `toolbox.individual()` to retrieve a single program. 

In [13]:
# configure the parameters
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

# Attribute generator
toolbox = base.Toolbox()
toolbox.register("expr_init", gp.genFull, pset=pset, min_=1, max_=2)

# Structure initializers
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr_init)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

Register the GP operators. They mate, mutate and change in various ways the individuals made of programs. Further individuals are the result from the GP algorithm using these GP operators.

In [14]:
toolbox.register("evaluate", evalRESTController)
toolbox.register("select", tools.selTournament, tournsize=7)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genHalfAndHalf, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

### Run the GP Algorithm

In [15]:
random.seed(69)

pop = toolbox.population(n=500)
hof = tools.HallOfFame(1)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", numpy.mean)
stats.register("std", numpy.std)
stats.register("min", numpy.min)
stats.register("max", numpy.max)

pop, log = algorithms.eaSimple(pop, toolbox, 0.5, 0.2, 2, stats, halloffame=hof)

gen	nevals	avg 	std	min 	max 
0  	500   	1999	0  	1999	1999
1  	325   	1999	0  	1999	1999
2  	303   	1999	0  	1999	1999


..x.
----------------------------------------------------------------------
Ran 4 tests in 0.380s

OK (expected failures=1)


<unittest.runner.TextTestResult run=4 errors=0 failures=0>

In [131]:
ind_read=ind

In [148]:
ind_create=ind

## Display and Plot

Plot the individual. It is a tree structure contained in the halloffame `hof` object.

`hof[0]` indicates the individual tree structure.

In [17]:
str(hof[0])

'if_then_else(read_cart, create_cart, update_cart)'

In [16]:
f=if_then_else(cps.create_cart, cps.create_cart, cps.read_cart)
f=prog2(cps.create_cart, cps.create_cart)
f=prog2(if_then_else(prog2(cps.update_cart, cps.update_cart), prog3(cps.create_cart, cps.create_cart, cps.create_cart), prog2(cps.create_cart, cps.update_cart)), cps.read_cart)
f=prog3(cps.read_cart, cps.read_cart, cps.create_cart)
f=prog3(cps.read_cart, cps.read_cart, cps.create_cart)
f=prog2(cps.update_cart, cps.create_cart)
f=if_then_else(cps.read_cart, cps.create_cart, cps.update_cart)
f=if_then_else(cps.create_cart,cps.create_cart,cps.read_cart)
f=if_then_else(cps.update_cart, cps.read_cart, cps.create_cart)
f=prog2(if_then_else(cps.create_cart, cps.read_cart, cps.read_cart), cps.update_cart)
f=prog2(cps.create_cart, cps.read_cart)
cps.reset()

In [50]:
c=return_cart_get(f)
c()

{'direction': 'left',
 'position': 0.7041449153009347,
 'velocity': 0.46315185174295925}

In [109]:
str(toolbox.individual())

'prog2(delete_cart, delete_cart)'

In [20]:
from deap import base, creator, gp

nodes, edges, labels = gp.graph(hof[0])

### Graphviz Section ###
import pygraphviz as pgv

g = pgv.AGraph()
g.add_nodes_from(nodes)
g.add_edges_from(edges)
g.layout(prog="dot")

for i in nodes:
    n = g.get_node(i)
    n.attr["label"] = labels[i]

g.draw('gp_controller.png')

![graph](gp_controller.png)

## Unit Testing the CartpoleServer

This unittest implements test methods for the CartpoleServer's CRUD functions.

In [92]:
import unittest
class Test_CartpoleServer(unittest.TestCase):
    # small test routine for the CartpoleServer
      
    def setUp(self):
        self.s=CartpoleServer
        
    def test_CartpoleServer(self):
        self.s.reset()

        c = self.s.read_cart()
        self.assertEqual(type(c), type(None))

        self.s.create_cart()
        c = self.s.read_cart()
        self.assertTrue( isinstance(type(c), type(Cart)) )

        c2 = self.s.read_cart()
        assert c == c2
        self.assertEqual(c.to_dict(), c2.to_dict())

        self.s.delete_cart()
        c = self.s.read_cart()
        self.assertEqual(type(c), type(None))

        self.s.create_cart()
        c = self.s.read_cart()
        self.assertTrue( isinstance(type(c), type(Cart)) )
        c2 = self.s.read_cart()
        self.assertEqual(c, c2)
        self.assertEqual(c.to_dict(), c2.to_dict())

        self.s.reset()
        c = self.s.read_cart()
        self.assertEqual(type(c), type(None))

In [94]:
suite = unittest.defaultTestLoader.loadTestsFromTestCase(Test_CartpoleServer)
unittest.TextTestRunner().run(suite)

.
----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

## Unit Testing the GP Controller Default Behavior

We devise a default test for the `GP_TestDefaultController`. The default behavior comprises calling the OpenAPI generated test methods from the super class `TestDefaultController`.   

In [17]:
import unittest
class Test_GP_TestDefaultController(unittest.TestCase):

    @classmethod
    def setUpClass(cls):
        super().setUpClass()
        cls.gp_test=GP_TestDefaultController()
        
    def setUp(self):
        self.gp_test=Test_GP_TestDefaultController.gp_test

    def test_default(self):
        gp_cart = self.gp_test.test_cart_get()

In [18]:
suite = unittest.defaultTestLoader.loadTestsFromTestCase(Test_GP_TestDefaultController)
unittest.TextTestRunner().run(suite)

.
----------------------------------------------------------------------
Ran 1 test in 0.174s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

## Unit Testing Monkey Patching

This section designs a few testcases for the `endpoint_config()` method in the `GP_TestDefaultController` class.

In [89]:
# functions replacing the default controller by Monkey Patching

from openapi_server.models import Cart

def testcase_all_correct():
    return Cart(position=1.1, velocity=2.2, direction='left')

def testcase_empty_return_object():
    return Cart()

def testcase_return_none():
    return None

def testcase_program_with_exception():
    raise Exception('Program raises an exception')

def testcase_return_string_type():
    return 'some string'

def testcase_return_obj_type():
    return type('obj', (object,), {'attribute' : 'attribute value'})

In [90]:
# the testcase with test methods

import unittest

class TestMonkeyPatching(unittest.TestCase):

    @classmethod
    def setUpClass(cls):
        super().setUpClass()
        cls.gp_test=GP_TestDefaultController()

    def setUp(self):
        self.url_path = '/api/v1/cart'
        self.gp_test=TestMonkeyPatching.gp_test
        
    def test_all_correct(self):
        ret = self.gp_test.endpoint_config(self.url_path, 'get', testcase_all_correct)
        self.assertTrue(ret)
        gp_cart = self.gp_test.test_cart_get()
    
    def test_empty_return_object(self):
        ret = self.gp_test.endpoint_config(self.url_path, 'get', testcase_empty_return_object)
        self.assertTrue(ret)
        gp_cart = self.gp_test.test_cart_get()
    
    @unittest.expectedFailure
    def test_return_none(self):
        ret = self.gp_test.endpoint_config(self.url_path, 'get', testcase_return_none)
        self.assertTrue(ret)
        gp_cart = self.gp_test.test_cart_get()

    @unittest.expectedFailure
    def test_program_with_exception(self):
        ret = self.gp_test.endpoint_config(self.url_path, 'get', testcase_program_with_exception)
        self.assertTrue(ret)
        gp_cart = self.gp_test.test_cart_get()
        
    @unittest.expectedFailure
    def test_return_string_type(self):
        ret = self.gp_test.endpoint_config(self.url_path, 'get', testcase_return_string_type)
        self.assertTrue(ret)
        gp_cart = self.gp_test.test_cart_get()

    @unittest.expectedFailure
    def test_return_obj_type(self):
        ret = self.gp_test.endpoint_config(self.url_path, 'get', testcase_return_obj_type)
        self.assertTrue(ret)
        gp_cart = self.gp_test.test_cart_get()


In [91]:
suite = unittest.defaultTestLoader.loadTestsFromTestCase(TestMonkeyPatching)
unittest.TextTestRunner().run(suite)

..xxxhttp://localhost/api/v1/cart validation error: 'some string' is not of type 'object'

Failed validating 'type' in schema:
    {'components': {'schemas': {'Cart': {'properties': {'direction': {'enum': ['left',
                                                                               'right'],
                                                                      'type': 'string'},
                                                        'position': {'format': 'double',
                                                                     'type': 'number'},
                                                        'velocity': {'format': 'double',
                                                                     'type': 'number'}},
                                         'type': 'object'},
                                'Direction': {'properties': {'direction': {'enum': ['left',
                                                                                    'right'],
       

<unittest.runner.TextTestResult run=6 errors=0 failures=0>

## Unit Testing DEAP for Finding Controller Functions

We test some behavior of DEAP to find GP controller functions.  

**Note:** This test does not run independently. It depends on the function definitions from above and asserts their existence.  

In [813]:
import unittest
import types

# cart_get() controller must return something
def return_cart_get(left):
    left()
    return cps.read_cart

class Test_DEAPController(unittest.TestCase):

    @classmethod
    def setUpClass(cls):
        super().setUpClass()
        assert isinstance(toolbox, base.Toolbox), 'toolbox not defined'
        cls.assertIsNotNone(toolbox.individual(), 'toolbox.individual() does not work')
        assert cps == CartpoleServer, 'cps is not a CartpoleServer'
    
    def setUp(self):
        self.assertIsInstance(pset, gp.PrimitiveSet, 'pset not defined')
        self.assertIsInstance(toolbox, base.Toolbox, 'toolbox not defined')
    
    def test_search_prog(self, n=100):
        
        def add_func2pset(expr, pset, func, num_args):
            pset_cloned = toolbox.clone(pset)
            pset_cloned.addPrimitive(func, num_args)
            prim_func = gp.Primitive(func.__name__, [object], object)
            expr_func = [prim_func] + expr
            return expr_func, pset_cloned
            
        function = None
        self.assertIsInstance(return_cart_get, types.FunctionType, 'return_cart_get() not defined')
        
        for i in range(n):
            cps.reset()
            # GP Program generation function 
            individual = toolbox.individual() # type: PrimitiveTree

            # expressions from a generated PrimitiveTree individual 
            expr = list(individual)            
            # add return_cart_get als new root
            expr_return, pset_return = add_func2pset(expr=expr,
                                                     pset=pset,
                                                     func=return_cart_get, 
                                                     num_args=1 )
            # rebuild tree from expressions 
            tree = gp.PrimitiveTree(expr_return)
            # .. and compile
            function = gp.compile(tree, pset_return)

            try:
                ret = function()
                if isinstance(ret, type(None)):
                    continue
            except Exception:
                continue
            break

        self.assertIsNotNone(function)
        self.assertIsInstance(function, types.MethodType)
            
        if not isinstance(ret, type(None)):    
            print('Loop: {}'.format(i))
            print('Function:\n{}'.format(str(tree)))
            print('\nReturn:\n{}'.format(ret))
        return function()
        

In [814]:
suite = unittest.defaultTestLoader.loadTestsFromTestCase(Test_DEAPController)
unittest.TextTestRunner().run(suite)

.

Loop: 5
Function:
return_cart_get(prog3(update_cart, update_cart, create_cart))

Return:
{'direction': 'right',
 'position': 0.012461126628846908,
 'velocity': 0.7247699102128887}



----------------------------------------------------------------------
Ran 1 test in 0.010s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

## Unit Testing `evaluate()`

**Observation:** Wiederholtes Aufrufen der compilierten Funktion führt zum selben Ergebnis während wiederholtes Compilieren der GP Controller Funktion zu verschiedenen Ergebnissen führt. Beide sollten jedoch das selbe Verhalten zeigen.

**Problem:** Ursache liegt in der `return_cart_get()` Funktion. Die Funktion führt das GP Controller Programm bereits zum Zeitpunkt aus, wenn die Routine zusammengebaut wird. Es wird nur noch die `cps.read` Routine zurückgegeben und als Test-Controller ausgeführt. Diese liefert dann ein wiederholt ein konstantes Ergebnis, da das GP Programm nicht mehr ausgeführt wird.

In [820]:
import unittest
import types


# cart_get() controller must return something
def return_cart_get(left):
    def run_first_return_second(f1,f2):
        f1()
        return f2()
    
    return partial(run_first_return_second, left, cps.read_cart)


class Test_Evaluate(unittest.TestCase):
        
    @classmethod
    def setUpClass(cls):
        super().setUpClass()
        assert cps == CartpoleServer, 'cps is not a CartpoleServer'
    
    def setUp(self):
        self.assertIsInstance(pset, gp.PrimitiveSet, 'pset not defined')
        self.assertIsInstance(return_cart_get, types.FunctionType, 'return_cart_get() not defined')
        
    def run_eval(self, gp_ctrl):
        ind = gp.PrimitiveTree.from_string(gp_ctrl, pset)
        fitness = evalRESTController(ind)[0]
        return fitness
        
    def test_create_controller(self):
        test_gp_controller = 'create_cart'
        fitness = self.run_eval(test_gp_controller)
        self.assertEqual(fitness, 5*200-1)
    
    @unittest.expectedFailure
    def test_read_controller(self):
        test_gp_controller = 'read_cart'
        fitness = self.run_eval(test_gp_controller)
        self.assertEqual(fitness, 5*200-1)

    def test_create_read_controller(self):
        test_gp_controller = 'prog2(create_cart, read_cart)'
        fitness = self.run_eval(test_gp_controller)
        self.assertEqual(fitness, 5*200-1)

    def test_read_create_controller(self):
        test_gp_controller = 'prog2(read_cart, create_cart)'
        fitness = self.run_eval(test_gp_controller)
        self.assertEqual(fitness, 5*200-1)                

In [821]:
suite = unittest.defaultTestLoader.loadTestsFromTestCase(Test_Evaluate)
unittest.TextTestRunner().run(suite)

..x.
----------------------------------------------------------------------
Ran 4 tests in 0.470s

OK (expected failures=1)


<unittest.runner.TextTestResult run=4 errors=0 failures=0>